This  just a testCase for Cal Corr for a given subject i from timeseries input data

The goal of this NoteBook file is to <br>
1. Create timeseries Feature tables (.csv) for each subject, labeled by actual area name
2. Calculate Correlation Matrix FC
3. Calculate also distance matrix (not used in the paper)
4. Save all Corrs and Dist matrices as tables with actual labels 

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
filepath = 'E:\AbideI\ProcessedData\\ex_file.csv'
ex_file_path = 'E:\\AbideI\\Outputs\\cpac\\nofilt_noglobal\\rois_aal\\Caltech_0051456_rois_aal.1D'
# For atlas conversion to csv (from atlasConverter)
aal_atlas_path = 'E:\\AbideI\\Atlases\\aal_labels.csv'
tt_atlas_path = 'E:\\AbideI\\Atlases\\tt_labels.csv'

In [ ]:
# Takes an input df, with names as in the standard downloaded labels csv, and convert column names
# atlas_df: df of first column contains labels, second contains brain regions
# atlas_df contains NO non-label rows. atlas_df contains prefix # value for each label
def parse_brain_regions(df, atlas_df= pd.read_csv(aal_atlas_path, skiprows=2, header=None)):
    if atlas_df is parse_brain_regions.__defaults__[0]:
        # For aal and tt atlas, if df is not passed, do this first. 
        atlas_df[0] = atlas_df[0].apply(lambda x: '#'+ str(x))
        print('note: aal atlas was used, hashed labels')
    out_df = df.copy()
    out_df.rename(columns=dict(zip(atlas_df[0], atlas_df[1])), inplace=True)
    return out_df

In [ ]:
# ReadAtalsFiles, Add preceeding hash first to atlas as in the label names
aal_atlas_df = pd.read_csv(aal_atlas_path, skiprows=2, header=None)
aal_atlas_df[0] = aal_atlas_df[0].apply(lambda x: '#'+ str(x))
tt_atlas_df = pd.read_csv(tt_atlas_path, skiprows=2, header=None) # First two rows contains non-atlas data. Check
tt_atlas_df[0] = tt_atlas_df[0].apply(lambda x: '#'+ str(x)) 

In [ ]:
def extractDataMats(corr_dict, diff_dict, feat_dict, nameslabelsdf):
    nameslabelsdf.set_index('subjectkey', inplace=True)

    # Create columns for correlation and SSD mats
    columns_diff_corr = []
    for k in range(nAreas):
        for j in range(k+1, nAreas):
            columns_diff_corr.append(f'{k}_{j}')
    columns_diff_corr.append('labels')
    corr_df = pd.DataFrame(None, index=nameslabelsdf.index, columns=columns_diff_corr)
    diff_df = pd.DataFrame(None, index=nameslabelsdf.index, columns=columns_diff_corr)

    # Create columns for feat mat
    columns_feat = []
    for i in range(nAreas):
        for j in feats_names:
            columns_feat.append(f'{i}_{j}')
    columns_feat.append('labels')
    feat_df = pd.DataFrame(None, index=nameslabelsdf.index, columns=columns_feat)

    # Start filling correlation and diff mats together
    # for key in nameslabelsdf.index:
    for key in corr_dict.keys():
        corr_mat = corr_dict[key]
        diff_mat = diff_dict[key]
        feat_mat = feat_dict[key]
        label = nameslabelsdf.loc[key, 'Dx']
        for ind in columns_diff_corr:
            if ind == 'labels':
                continue
            a1, a2 = ind.split('_')
            a1 = int(a1); a2=int(a2)
            corr_df.loc[key,ind] = corr_mat[a1, a2]
            diff_df.loc[key,ind] = diff_mat[a1, a2]
        corr_df.loc[key,'labels'] = label
        diff_df.loc[key,'labels'] = label

        feat_df.loc[key, :-1] = feat_mat.ravel()
        feat_df.loc[key,'labels'] = label

    return corr_df, diff_df, feat_df

In [ ]:
# Creating correlation matrix for a single file, written as csv
def CreateCorr(filepath):
    data_df = pd.read_csv(filepath)
    nAreas = data_df.shape[1]
    temp_corr = np.zeros((nAreas, nAreas))
    temp_diff = np.zeros_like(temp_corr)
    col_nam=[]
    for i in range(nAreas):
        temp_corr[i,i] = 1
        temp_diff[i,i] = 0
        for j in range(i+1, nAreas):
            temp_corr[i,j] = np.corrcoef(data_df.iloc[:,i], data_df.iloc[:,j])[0,1] # It returns the covar matrix of 2x2
            temp_corr[j,i] = temp_corr[i,j]
            temp_diff[i,j] = np.sqrt(np.sum(np.square(data_df.iloc[:,i]-data_df.iloc[:,j])))
            temp_diff[j,i] = temp_diff[i,j] 
#             col_nam.append( '__'.join([df.columns[j] ,df.columns[i]]) )
    temp_corr = np.nan_to_num(temp_corr)
    temp_diff = np.nan_to_num(temp_diff)
    return temp_corr, temp_diff            # col_nam

In [ ]:
def create_corr_names(col_names):
    col_nam = []
    n_areas = len(col_names)
    for i in range(n_areas):
        for j in range(i+1, n_areas):
            col_nam.append( '__'.join([col_names[i].replace(' ', '') ,col_names[j].replace(' ', '')] ) )
    return col_nam

In [ ]:
def create_feat_matrices(FolderPath, atlas_df, output_path = 'E:\\AbideI\\ProcessedData\\nofilt_noglobal_rois_aal\\feat\\', atl_name='aal'):
    subjs_fldrs = [os.path.join(FolderPath, f) for f in os.listdir(FolderPath)]
    for filepath in subjs_fldrs:
        subj = filepath.split('\\')[-1].split('.')[0]
        input_df_aal = pd.read_csv(filepath, header=0, sep='\t')
        parse_brain_regions(input_df_aal,atlas_df).to_csv(output_path + subj+ '.csv', index=False)

In [ ]:
def create_corr_matrices(FolderPath, col_names, output_path = 'E:\\AbideI\\ProcessedData\\nofilt_noglobal_rois_aal\\corr\\'):
    corr_col_names = create_corr_names(col_names)
    subjects_corr_df = pd.DataFrame(columns = corr_col_names)
    subjects_dist_df = pd.DataFrame(columns = corr_col_names)
    subjs_fldrs = [os.path.join(FolderPath, f) for f in os.listdir(FolderPath)]
    for filepath in subjs_fldrs:
        subj = filepath.split('\\')[-1].split('.')[0]
#         if os.path.exists(output_path + subj+ '_corr.csv'): #unhash if needed
#             print(f'Subj %s already exists, continuing') 
#             continue
        sub_name = ('_').join(subj.split('_')[:2])
        print('working on Subj: ', subj)
        corr, dist = CreateCorr(filepath)
        # Save each file independently:
        pd.DataFrame(data=corr, columns = col_names).to_csv(output_path + subj+ '_corr.csv',index = False)
        pd.DataFrame(data=dist, columns = col_names).to_csv(output_path + subj+ '_dist.csv',index = False)
        corr_vals = corr[np.triu_indices_from(corr, k=1)]
        dist_vals = dist[np.triu_indices_from(dist, k=1)]
        subjects_corr_df.loc[subj,:] = corr_vals.reshape((1,-1))
        subjects_dist_df.loc[subj,:] = dist_vals.reshape((1,-1))
#         if subjects_dist_df.shape[0]==5:#debug only
#             subjects_corr_df
    subjects_corr_df.to_csv(output_path + '_CORR.csv',index = True)
    subjects_dist_df.to_csv(output_path + '_DIST.csv',index = True)
    return subjects_corr_df, subjects_dist_df

In [ ]:
main_path = 'E:\\AbideI\\Outputs\\cpac\\'
sub_pth_tt = 'Caltech_0051456_rois_tt.1D'
sub_pth_aal = 'Caltech_0051456_rois_aal.1D'
out_pth = 'E:\\AbideI\\ProcessedData\\'

In [ ]:
# Now, Working on 1: nonfilt nonglob aal atlas:
filt_pth = 'nofilt_noglobal\\'
atl_pth = 'rois_aal\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_aal), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, aal_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), aal_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df, diff_df = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

In [ ]:
# Now, Working on 2: nonfilt nonglob tt atlas:
filt_pth = 'nofilt_noglobal\\'
atl_pth = 'rois_tt\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_tt), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, tt_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), tt_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df2, diff_df2 = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

In [ ]:
# Now, Working on 3: nofilt_global aal atlas:
filt_pth = 'nofilt_global\\'
atl_pth = 'rois_aal\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_aal), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, aal_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), aal_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df3, diff_df3 = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

In [ ]:
# Now, Working on 4: nofilt_global tt atlas:
filt_pth = 'nofilt_global\\'
atl_pth = 'rois_tt\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_tt), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, tt_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), tt_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df4, diff_df4 = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

In [ ]:
# Now, Working on 5: filt_noglobal aal atlas:
filt_pth = 'filt_noglobal\\'
atl_pth = 'rois_aal\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_aal), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, aal_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), aal_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df5, diff_df5 = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

In [ ]:
# Now, Working on 6: filt_noglobal tt atlas:
filt_pth = 'filt_noglobal\\'
atl_pth = 'rois_tt\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_tt), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, tt_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), tt_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df6, diff_df6 = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

In [ ]:
# Now, Working on 7: filt_global aal atlas:
filt_pth = 'filt_global\\'
atl_pth = 'rois_aal\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_aal), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, aal_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), aal_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df7, diff_df7 = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

In [ ]:
# Now, Working on 8: filt_global tt atlas:
filt_pth = 'filt_global\\'
atl_pth = 'rois_tt\\'
mrgd_pth = filt_pth.strip('\\') + '_' + atl_pth
ex_df = pd.read_csv((main_path+filt_pth+atl_pth+sub_pth_tt), header=0, sep='\t')
colnames = parse_brain_regions(ex_df, tt_atlas_df).columns
create_feat_matrices((main_path+filt_pth+atl_pth), tt_atlas_df, output_path = out_pth+mrgd_pth+'feat\\')
corr_df8, diff_df8 = create_corr_matrices(out_pth+mrgd_pth+'feat\\', colnames, output_path =out_pth+mrgd_pth+'corr\\' )

Algortithm<br>
I) Read Each subject values, and convert area names<br>
II) Calc Corr, Diff, and other useful metrics that may encode the features. [signal representatives] <br>
III) Then, For each Feature type
1) Create column names, then create empty df <br>
2) for each index (Subject name) add to each subject his feature values. <br>
3) Save two csv files, indexed by subject names.<br>
- Subject name is [:-2], where some has middle identifiers, better remove underscore for consistent naming